1. 전처리를 잘 해줬지만 답변이 나오지 않는 이유는 질문이 잘못되었기 때문.
2. 직장인과 거주자의 연관성이 부족했음.
3. 그래서 직장인을 거주자로 바꿔 줄 수 있는 LCEL(LangChain Expression Language)

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [3]:
import os
import time

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name='tax-index' #tax-markdown-index

pc=Pinecone(api_key=pinecone_api_key)

#database=PineconeVectorStore.from_documents(document_list,embedding,index_name=index_name)

# 먼저 database 객체 생성
database = PineconeVectorStore(index_name=index_name, embedding=embedding)

# documents를 나누어 업로드 (예: 100개씩 배치 처리)
batch_size = 100
for i in range(0, len(document_list), batch_size):
    batch = document_list[i:i + batch_size]
    database.add_documents(batch)

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [5]:
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='ac970249-4fe0-48f2-8f3c-e6a77748b663', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
#prompt
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [8]:
ai_message = qa_chain.invoke({"query": query})

In [9]:
ai_message


{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 기본세율에 따라 다르게 적용될 수 있습니다. 구체적인 세액은 소득공제 및 세액공제를 고려해야 하므로 정확한 금액을 계산하기 위해서는 추가 정보가 필요합니다. 따라서 소득세는 단순하게 정해진 수치로는 제공할 수 없습니다.'}

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [11]:
new_question = dictionary_chain.invoke({"question": query})
query


'연봉 5천만원인 직장인의 소득세는 얼마인가요?'

In [12]:
new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [ ]:
ai_response = tax_chain.invoke({"question": query})
ai_response


{'query': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 거주자의 소득세는 84만원에 (5천만원 - 1,400만원)의 15%를 더한 금액입니다. 즉, 소득세는 84만원 + (3,600만원 × 0.15) = 84만원 + 540만원 = 624만원입니다. 따라서, 총 소득세는 624만원입니다.'}